In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from keras.datasets import fashion_mnist
from keras.models import Sequential ,Model
from keras.layers import Dense,Dropout,Input
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

In [4]:
def load_data():
    (xtrain,ytrain),(xtest,ytest) = fashion_mnist.load_data()
    xtrain = (xtrain.astype(np.float32)-127.5/127.5)
    xtrain = xtrain.reshape(60000,784)
    return xtrain,ytrain,xtest,ytest
xtrain,ytrain,xtest,ytest = load_data()
print(xtrain.shape)

(60000, 784)


In [5]:
def adam_optimizer():
    return Adam(lr=0.01)

In [15]:
def create_generator():
    generator = Sequential()
    generator.add(Dense(256,input_dim = 100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(784,activation='tanh'))
    generator.compile(loss='binary_crossentropy',optimizer=adam_optimizer(),metrics=['accuracy'])
    
    generator.summary()
    return generator

g = create_generator()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 784)              

In [16]:
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(1024,input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
                      
    discriminator.add(Dense(1,activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy',optimizer=adam_optimizer(),metrics=['accuracy'])
    return discriminator
d = create_discriminator()
d.summary()                     

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)              

In [21]:
def create_gan(discriminator,generator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs = gan_input,outputs = gan_output)
    gan.compile(loss='binary_crossentropy',optimizer='adam')
    return gan
gan = create_gan(d,g)
gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_8 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_9 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


In [39]:
def plot_genrated_images(epoch,generator,examples=100,dim=(10,10),figsize=(10,10)):
    noise = np.random.normal(loc=0,scale=1,size=[examples,100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)

In [ ]:
def training(epochs=1,batch_size=128):
    (xtrain,ytrain,xtest,ytest) = load_data()
    batch_count = xtrain.shape[0]/batch_size
    
    generator = create_generator()
    discriminator = create_discriminator()
    gan = create_gan(discriminator,generator)
    
    for e in range(1,epochs+1):
        print("epoch %d"%e)
        for _ in tqdm(range(batch_size)):
            noise = np.random.normal(0,1,[batch_size,100])
            generated_images = generator.predict(noise)
            image_batch = xtrain[np.random.randint(low=0,high=xtrain.shape[0],size=batch_size)]
            x = np.concatenate([image_batch,generated_images])
            
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9
            
            discriminator.trainable = True
            discriminator.train_on_batch(x,y_dis)
            
            noise = np.random.normal(0,1,[batch_size,100])
            y_gen = np.ones(batch_size)
          
            discriminator.trainable = False
            gan.train_on_batch(noise,y_gen)
            
        if e%100==0:
            plot_generated_images(e,generator)
training(200,128)     

  0%|          | 0/128 [00:00<?, ?it/s]

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_20 (LeakyReLU)   (None, 1024)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 784)             

  1%|          | 1/128 [00:00<00:13,  9.50it/s]

epoch 2


  1%|          | 1/128 [00:00<00:18,  6.93it/s]

...

  1%|          | 1/128 [00:00<00:16,  7.56it/s]

epoch 14


  1%|          | 1/128 [00:00<00:14,  8.52it/s]

epoch 15


  5%|▍         | 6/128 [00:00<00:15,  8.02it/s]